# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [16]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [17]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [18]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-20 14:25:04,722 [INFO] General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
2025-10-20 14:25:04,723 [INFO] ✔ Completed in 0.00016379356384277344 seconds
2025-10-20 14:25:04,723 [INFO] Binding consume in domain (solar, l, q)
2025-10-20 14:25:04,724 [INFO] ✔ Completed in 0.0004134178161621094 seconds
2025-10-20 14:25:04,725 [INFO] General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
2025-10-20 14:25:04,725 [INFO] ✔ Completed in 0.00010609626770019531 seconds
2025-10-20 14:25:04,725 [INFO] Binding consume in domain (wind, l, y)
2025-10-20 14:25:04,726 [INFO] ✔ Completed in 6.937980651855469e-05 seconds
2025-10-20 14:25:04,726 [INFO] General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
2025-10-20 14:25:04,727 [INFO] ✔ Completed in 0.00023031234741210938 seconds
2025-10-20 14:25:04,728 [INFO] Binding release in domain (power, l, q)
2025-10-20 14:25:

aaaaa solar consume
aaaaa wind consume
aaaaa power release


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [19]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-20 14:25:04,734 [INFO] Binding capacity in domain (wf, l, y)
2025-10-20 14:25:04,735 [INFO] ✔ Completed in 0.00018024444580078125 seconds
2025-10-20 14:25:04,735 [INFO] Binding capacity in domain (wf, l, y)
2025-10-20 14:25:04,735 [INFO] ✔ Completed in 0.00015282630920410156 seconds


aaaaa wf capacity
aaaaa wf capacity


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [20]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-20 14:25:04,743 [INFO] Binding operate in domain (wf, l, q)
2025-10-20 14:25:04,744 [INFO] ✔ Completed in 0.00032067298889160156 seconds
2025-10-20 14:25:04,745 [INFO] Mapping operate: (wf, l, q) → (wf, l, y)
2025-10-20 14:25:04,745 [INFO] ✔ Completed in 0.000s


aaaaa wf operate
aaaaa wf capacity
aaaaa usd spend
aaaaa wf operate
aaaaa usd spend


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [21]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-20 14:25:04,761 [INFO] Binding capacity in domain (pv, l, y)
2025-10-20 14:25:04,761 [INFO] ✔ Completed in 0.00016808509826660156 seconds
2025-10-20 14:25:04,762 [INFO] Binding capacity in domain (pv, l, y)
2025-10-20 14:25:04,763 [INFO] ✔ Completed in 0.00031375885009765625 seconds
2025-10-20 14:25:04,764 [INFO] Binding operate in domain (pv, l, q)
2025-10-20 14:25:04,764 [INFO] ✔ Completed in 0.0001914501190185547 seconds
2025-10-20 14:25:04,766 [INFO] Mapping operate: (pv, l, q) → (pv, l, y)
2025-10-20 14:25:04,766 [INFO] ✔ Completed in 0.000s


aaaaa pv capacity
aaaaa pv capacity
aaaaa pv operate
aaaaa pv capacity
aaaaa usd spend
aaaaa pv operate
aaaaa usd spend


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [22]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-20 14:25:04,774 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-20 14:25:04,775 [INFO] ✔ Completed in 0.0002548694610595703 seconds
2025-10-20 14:25:04,776 [INFO] Binding invcapacity in domain (power.lii, l, y)
2025-10-20 14:25:04,776 [INFO] ✔ Completed in 0.00014829635620117188 seconds
2025-10-20 14:25:04,778 [INFO] General Resource Balance for power.lii in (l, y): initializing constraint, adding inventory(power.lii, l, y)
2025-10-20 14:25:04,778 [INFO] Binding capacity in domain (lii.charge, l, y)
2025-10-20 14:25:04,779 [INFO] ✔ Completed in 7.414817810058594e-05 seconds
2025-10-20 14:25:04,779 [INFO] Binding capacity in domain (lii.discharge, l, y)
2025-10-20 14:25:04,780 [INFO] ✔ Completed in 7.200241088867188e-05 seconds


aaaaa power.lii invcapacity
aaaaa power.lii invcapacity
aaaaa power.lii invcapacity
aaaaa usd spend
aaaaa power.lii inventory
aaaaa usd spend
aaaaa lii.charge capacity
aaaaa lii.discharge capacity


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [23]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-20 14:25:04,785 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
2025-10-20 14:25:04,786 [INFO] ✔ Completed in 0.00019431114196777344 seconds
2025-10-20 14:25:04,787 [INFO] General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
2025-10-20 14:25:04,787 [INFO] ✔ Completed in 0.00028443336486816406 seconds
2025-10-20 14:25:04,789 [INFO] General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
2025-10-20 14:25:04,789 [INFO] ✔ Completed in 0.00014710426330566406 seconds
2025-10-20 14:25:04,790 [INFO] General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
2025-10-20 14:25:04,791 [INFO] ✔ Completed in 0.000171661376953125 seconds
2025-10-20 14:25:04,792 [INFO] Assuming  power.lii inventory capacity is unbounded in (l, y)
2025-10-20 14:25:04,792 [INFO] Assuming inventory of power.lii is bound by inventory capacity in (l, y)
2025-10-20 14:25:04,793 [INFO

aaaaa power produce
aaaaa pv operate
aaaaa pv operate
aaaaa solar expend
aaaaa pv operate
aaaaa pv operate
aaaaa pv capacity
aaaaa power produce
aaaaa wf operate
aaaaa wf operate
aaaaa wind expend
aaaaa wf operate
aaaaa wf operate
aaaaa wf capacity
aaaaa power.lii invcapacity
aaaaa power.lii inventory
aaaaa lii.charge operate
aaaaa power.lii produce
aaaaa lii.charge operate
aaaaa lii.charge operate
aaaaa power expend
aaaaa lii.charge operate
aaaaa lii.charge operate
aaaaa lii.charge capacity
aaaaa lii.discharge operate
aaaaa power produce
aaaaa lii.discharge operate
aaaaa lii.discharge operate


KeyError: (power, l, q[0], operate, pv, operate, wf, operate, lii.discharge)

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [25]:
m.grb[m.solar][m.l]

defaultdict(list,
            {q: [consume(solar, l, q), expend(solar, l, q, operate, pv)]})

In [32]:
m.processes

[wf, pv, lii.charge, lii.discharge]

In [35]:
m.periods

[q, y]

In [28]:
m.dispositions[m.operate][m.domain]

AttributeError: example2 has no 'domain'

In [ ]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.sol()

'Use .opt() to generate solution'

## Optimize!

In [ ]:
m.show()

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.usd.spend.opt()

2025-10-20 11:13:30,395 [INFO] Mapping spend: (usd, l, y, capacity, wf) → (usd, l, y)
2025-10-20 11:13:30,395 [INFO] ✔ Completed in 0.000s
2025-10-20 11:13:30,396 [INFO] Mapping spend: (usd, l, y, operate, wf) → (usd, l, y)
2025-10-20 11:13:30,396 [INFO] ✔ Completed in 0.000s
2025-10-20 11:13:30,397 [INFO] Mapping spend: (usd, l, y, capacity, pv) → (usd, l, y)
2025-10-20 11:13:30,397 [INFO] ✔ Completed in 0.000s
2025-10-20 11:13:30,397 [INFO] Mapping spend: (usd, l, y, operate, pv) → (usd, l, y)
2025-10-20 11:13:30,398 [INFO] ✔ Completed in 0.000s
2025-10-20 11:13:30,398 [INFO] Mapping spend: (usd, l, y, invcapacity, power.lii) → (usd, l, y)
2025-10-20 11:13:30,399 [INFO] ✔ Completed in 0.000s
2025-10-20 11:13:30,399 [INFO] Mapping spend: (usd, l, y, inventory, power.lii) → (usd, l, y)
2025-10-20 11:13:30,399 [INFO] ✔ Completed in 0.000s
2025-10-20 11:13:30,400 [INFO] Generating Program(example2).mps
2025-10-20 11:13:30,405 [INFO] Creating gurobi model for Program(example2)


Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros


2025-10-20 11:13:30,407 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-20 11:13:30,417 [INFO] Solution found. Use .sol() to display it
2025-10-20 11:13:30,417 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [ ]:
m.inventory.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [ ]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [ ]:
m.produce(m.power, m.lii.discharge.operate, m.q).sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [ ]:
m.capacity.reporting.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
m.capacity.sol()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>